In [1]:
import time
import pytz
import pandas as pd

from itertools import chain
from datetime import timedelta, date, datetime, timezone
from basketball_reference_web_scraper import client

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from secrets import *

In [2]:
dbname = 'cluj'
engine = create_engine('postgres://%s:%s@localhost/%s'%(AUTH['db_user'],AUTH['db_pass'],dbname))

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


### Pull yesterdays boxscores

In [3]:
# need to create a primary key, maybe name+date+team string?

In [22]:
day = datetime.now() - timedelta(days=1)
print(day)
boxscores = client.player_box_scores(day=4, month=2, year=2020)

2020-02-04 08:03:58.042655


In [23]:
for item in boxscores:
    item.update( {"date":datetime.strftime(day.date(), format = '%Y-%m-%d')})

In [6]:
boxscores_df = pd.DataFrame(boxscores)
boxscores_df['rebounds'] = boxscores_df.offensive_rebounds + boxscores_df.defensive_rebounds
boxscores_df.rename(columns={'attempted_field_goals':'fga', 'attempted_free_throws':'fta',
                            'made_three_point_field_goals':'threes', 'made_field_goals':'fgm',
                            'made_free_throws':'ftm'}, inplace=True)
boxscores_df['twos'] = boxscores_df.fgm - boxscores_df.threes
boxscores_df['points'] = (boxscores_df.threes * 3) + (boxscores_df.twos * 2) + (boxscores_df.ftm * 1)
boxscores_df.drop(columns=['attempted_three_point_field_goals','defensive_rebounds','offensive_rebounds',
                          'game_score','slug','turnovers','outcome','twos','personal_fouls','location'], inplace=True)
boxscores_df['opponent'] = boxscores_df.opponent.apply(lambda x: x.name)
boxscores_df['team'] = boxscores_df.team.apply(lambda x: x.name)
boxscores_df.columns.values

AttributeError: 'DataFrame' object has no attribute 'offensive_rebounds'

In [7]:
boxscores_df.to_sql('boxscores', con=engine, if_exists='append', index=False)